In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import xgboost as xgb
import sklearn.metrics as metrics
import datetime

In [2]:
inputfolder = 'data'
datadf = pd.read_csv(os.path.join(inputfolder,"final_match_data.csv"))
datadf['datetime'] = pd.to_datetime(datadf['datetime'])
with open(os.path.join(inputfolder,'datacolumns.txt'),'r') as f:
    content = f.readlines()
datacols = [x.strip() for x in content]
print(datadf.dtypes)
display(datadf.head())

atpdata = pd.read_csv(os.path.join(inputfolder,"ATP_matches.csv"))
cols = atpdata.drop(['Winner','Loser','Tournament','Tournament_Date','Court_Surface','Round_Description'],axis=1).columns
for col in cols:
    atpdata.loc[:,col]=pd.to_numeric(atpdata[col],errors='coerce')
print(atpdata.dtypes)
atpdata['datetime'] = pd.to_datetime(atpdata['Tournament_Date'])
atpdata['year'] = pd.DatetimeIndex(atpdata['datetime']).year
def fillrank(row,col,ATPframe):
    if np.isnan(row[col]):
            year = row['year']
            tournament = row['Tournament']
            surface = row['Court_Surface']
            yeardf = ATPframe[ATPframe['year']==year] #i can do this slicing in one line, but im doing in two to allow for easier debugging
            tourndf = ATPframe[(ATPframe['Tournament']==tournament)&(ATPframe['Court_Surface']==surface)]
            maxrank = int(max([tourndf['Winner_Rank'].max(),tourndf['Loser_Rank'].max()]))
            return maxrank+1
    else:
        return row[col]
atpdata.loc[:,'Winner_Rank'] = atpdata.apply(fillrank,axis=1,args=('Winner_Rank',atpdata))
atpdata.loc[:,'Loser_Rank'] = atpdata.apply(fillrank,axis=1,args=('Loser_Rank',atpdata))

display(atpdata.head())

#only going to use matches from 2014 onwards, not ussing matches from 2012-13 so we have enough historic data for matches in 2014
atpHardOnly = atpdata.loc[(atpdata['Court_Surface']=='Hard')&(atpdata['year']>2013),['Winner','Loser','datetime']]
frame1,frame2 = train_test_split(atpHardOnly,test_size=.5)
frame1.columns = ['Player1','Player2','datetime']
frame1['Player1Win'] = 1

frame2.columns = ['Player2','Player1','datetime']
frame2['Player1Win'] = 0

traininput = pd.concat([frame1,frame2])
traininput.sort_index(inplace=True)
print(traininput['Player1Win'].mean())
display(traininput.head())

# splitting into X and Y
X = traininput.drop('Player1Win',axis=1)
Y = traininput['Player1Win']

Player                        object
Rank                         float64
Sets_Won                     float64
Games_Won                      int64
Aces                         float64
DoubleFaults                 float64
FirstServes_Won              float64
FirstServes_In               float64
SecondServes_Won             float64
SecondServes_In              float64
BreakPoints_Won              float64
BreakPoints                  float64
ReturnPoints_Won             float64
ReturnPoints_Faced           float64
TotalPoints_Won                int64
won_game?                      int64
FirstServes_ratio            float64
SecondServes_ratio           float64
BreakPoints_ratio            float64
ReturnPoints_ratio           float64
datetime              datetime64[ns]
year                           int64
Tournament                    object
Round_Description             object
Court_Surface                 object
Total_Serves                 float64
Aces%                        float64
S

,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,...,BreakPoints_ratio,ReturnPoints_ratio,datetime,year,Tournament,Round_Description,Court_Surface,Total_Serves,Aces%,ServesWon%
0,Edouard Roger-Vasselin,106.0,2.0,12,5.0,2.0,22.0,30.0,12.0,19.0,...,0.571429,0.423729,2012-01-02,2012,Chennai,First Round,Hard,49.0,0.102041,0.693878
1,Dudi Sela,83.0,2.0,12,2.0,0.0,14.0,17.0,11.0,16.0,...,0.428571,0.620690,2012-01-02,2012,Chennai,First Round,Hard,33.0,0.060606,0.757576
2,Go Soeda,120.0,2.0,19,6.0,1.0,48.0,64.0,19.0,39.0,...,0.454545,0.400000,2012-01-02,2012,Chennai,First Round,Hard,103.0,0.058252,0.650485
3,Yuki Bhambri,345.0,2.0,12,1.0,2.0,22.0,29.0,9.0,17.0,...,0.384615,0.548387,2012-01-02,2012,Chennai,First Round,Hard,46.0,0.021739,0.673913
4,Yuichi Sugita,235.0,2.0,12,3.0,1.0,37.0,51.0,11.0,27.0,...,0.428571,0.407407,2012-01-02,2012,Chennai,First Round,Hard,78.0,0.038462,0.615385


C:\Users\Hugh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (11,12,13,14,17,18,24,25,26,27,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Winner                        object
Loser                         object
Tournament                    object
Tournament_Date               object
Court_Surface                 object
Round_Description             object
Winner_Rank                  float64
Loser_Rank                   float64
Retirement_Ind                 int64
Winner_Sets_Won              float64
Winner_Games_Won               int64
Winner_Aces                  float64
Winner_DoubleFaults          float64
Winner_FirstServes_Won       float64
Winner_FirstServes_In        float64
Winner_SecondServes_Won      float64
Winner_SecondServes_In       float64
Winner_BreakPoints_Won       float64
Winner_BreakPoints           float64
Winner_ReturnPoints_Won      float64
Winner_ReturnPoints_Faced    float64
Winner_TotalPoints_Won         int64
Loser_Sets_Won               float64
Loser_Games_Won                int64
Loser_Aces                   float64
Loser_DoubleFaults           float64
Loser_FirstServes_Won        float64
L

,Winner,Loser,Tournament,Tournament_Date,Court_Surface,Round_Description,Winner_Rank,Loser_Rank,Retirement_Ind,Winner_Sets_Won,...,Loser_FirstServes_In,Loser_SecondServes_Won,Loser_SecondServes_In,Loser_BreakPoints_Won,Loser_BreakPoints,Loser_ReturnPoints_Won,Loser_ReturnPoints_Faced,Loser_TotalPoints_Won,datetime,year
0,Edouard Roger-Vasselin,Eric Prodon,Chennai,02-Jan-12,Hard,First Round,106.0,97.0,0,2.0,...,33.0,13.0,26.0,1.0,3.0,15.0,49.0,49,2012-01-02,2012
1,Dudi Sela,Fabio Fognini,Chennai,02-Jan-12,Hard,First Round,83.0,48.0,0,2.0,...,32.0,5.0,26.0,0.0,1.0,8.0,33.0,30,2012-01-02,2012
2,Go Soeda,Frederico Gil,Chennai,02-Jan-12,Hard,First Round,120.0,102.0,0,2.0,...,70.0,18.0,35.0,2.0,4.0,36.0,103.0,99,2012-01-02,2012
3,Yuki Bhambri,Karol Beck,Chennai,02-Jan-12,Hard,First Round,345.0,101.0,0,2.0,...,33.0,13.0,29.0,2.0,3.0,15.0,46.0,43,2012-01-02,2012
4,Yuichi Sugita,Olivier Rochus,Chennai,02-Jan-12,Hard,First Round,235.0,67.0,0,2.0,...,32.0,13.0,22.0,1.0,7.0,30.0,78.0,62,2012-01-02,2012


0.4999257609502598


C:\Users\Hugh\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Hugh\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Hugh\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=Fals

,Player1,Player1Win,Player2,datetime
5893,Albert Ramos-Vinolas,0,Edouard Roger-Vasselin,2014-01-01
5894,Aleksandr Nedovyesov,0,Guillermo Garcia-Lopez,2014-01-01
5895,Henri Laaksonen,0,Aljaz Bedene,2014-01-01
5896,Jeevan Nedunchezhiyan,0,Jiri Vesely,2014-01-01
5897,Benjamin Becker,1,Julian Reister,2014-01-01


In [3]:
#defining preprocessing functions (from wrangleDataAttempt2)
def days_difference(date1,date2):
            diff = date2-date1
            return diff.days

def get_player_stats(inputframe,harddf,datacols,playercol):
    #idea is to pass a panda dataframe with columns [playername, tournamentdate] and be able to return a dataframe with the stats
    length = inputframe.shape[0]
    historicnames = ['careeravg_'+x for x in datacols]
    ytdnames = ['ytd_'+x for x in datacols]
    colnames = [playercol,'datetime','Rank','ytd_Aces%_Stddev','ytd_ServesWon%_Stddev'] + historicnames+ytdnames
    outputframe = pd.DataFrame(index=range(0,length),columns=colnames)
    outputframe.loc[:,playercol] = inputframe[playercol]
    outputframe.loc[:,'datetime'] = inputframe['datetime']
    for index in range(0,length):
        #extract player name and date of game
        row = inputframe.iloc[index,:]
        playername = row[playercol]
        date = row['datetime']
        
        #grab only data for that player before that date
        tempdf = harddf[harddf['Player']==playername]
        tempdf.loc[:,'timedelta'] = tempdf['datetime'].apply(days_difference,args=(date,))
        tempdf = tempdf[tempdf['timedelta']>0]
        if tempdf.empty:
            continue
        ytddf = tempdf.loc[tempdf['timedelta']<=365]
#         display(ytddf.head())
        currank = tempdf.loc[tempdf['timedelta'].idxmin(),'Rank']
        historicframe = tempdf.loc[:,datacols].mean()
        historicframe.index = historicnames
        YTDframe = ytddf.loc[:,datacols].mean()
        YTDframe.index = ytdnames
        
        #code for debugging NaNs
#         if tempdf.isna().sum().sum()>0:
#             print(playername)
#             print("ytdframe shape: {}, missing values: {}".format(ytddf.shape,ytddf.isna().sum().sum()))
#             print(ytddf.isna().sum())
#             print("careerdf shape: {}, missing values: {}".format(tempdf.shape,tempdf.isna().sum().sum()))
#             print(tempdf.isna().sum())

        outputframe.loc[index,['Rank','ytd_Aces%_Stddev','ytd_ServesWon%_Stddev']] = [currank,ytddf['Aces%'].std(),ytddf['ServesWon%'].std()]
        outputframe.loc[index,historicnames]=historicframe
        outputframe.loc[index,ytdnames]=YTDframe
    return outputframe

def get_difference(frame1,frame2,colname=None, prefix1='',prefix2='',index1=None,index2=None):
    #check if index between frames are identical, if not, reset both
    idx_check = frame1.index.equals(frame2.index)
    if not(idx_check):
        frame1.reset_index(drop=True, inplace=True)
        frame2.reset_index(drop=True, inplace=True)
    if colname is None:
        colname = frame1.columns
    if frame1.shape[0] != frame2.shape[0]:
        raise ValueError('both frames must contain the same number of rows/n Frame1: %{}/mn Frame2: %{}'.format(frame1.shape[0],frame2.shape[0]))
    tempdiffcolnames = [x+"_diff" for x in colname]
    diffcolnames = list()
    if index1 is not None:
        diffcolnames.append(index1)
    if index2 is not None:
        diffcolnames.append(index2)
    diffcolnames = diffcolnames + tempdiffcolnames
    if idx_check:
        difframe = pd.DataFrame(index=frame1.index,columns=diffcolnames,data=0)
    else:
        difframe = pd.DataFrame(index=range(0,frame1.shape[0]),columns=diffcolnames,data=0)
    if index1 is not None:
        difframe[index1] = frame1[index1]
    if index2 is not None:
        difframe[index2] = frame2[index2]
    for col in colname:
        frame1name = prefix1+col
        frame2name = prefix2+col
        difframename = col+"_diff"
        difframe[difframename]=frame1[frame1name] - frame2[frame2name]
    return difframe

def convert_to_data(inputframe,matchdata,datacols,player1colname="Player1",player2colname="Player2"):
    inputframe.reset_index(drop=True,inplace=True)
    
    player1frame = inputframe.loc[:,[player1colname,'datetime']]
    player2frame = inputframe.loc[:,[player2colname,'datetime']]
    
    player1df = get_player_stats(player1frame,matchdata,datacols,player1colname)
    player2df = get_player_stats(player2frame,matchdata,datacols,player2colname)
    
    #append columns indicating whether this is a players first recorded Hard surface ATPgame (if it's their first game, Rank will return NaNs)
    player1df['Player1fg'] = player1df['Rank'].isna()
    player2df['Player2fg'] = player2df['Rank'].isna()
    
#     player1df = player1df[~player1df['Player1fg']].fillna(0)
#     player2df = player2df[~player2df['Player2fg']].fillna(0)
#     if player1df.isna().sum().sum() >0:
#         display(player1df)
    
    
    #for debugging purposes
    if player1df.shape[0] != player2df.shape[0]:
        return (player1df,player2df)
    
    newdatacols = player1df.drop([player1colname,'datetime','Player1fg'],axis=1).columns
    outputdf = get_difference(player1df,player2df,newdatacols,index1=player1colname,index2=player2colname)
    outputdf.loc[:,'Player1fg'] = player1df['Player1fg']
    outputdf.loc[:,'Player2fg'] = player2df['Player2fg']
    outputdf.loc[:,'datetime'] = player1df['datetime']
    
    return outputdf.infer_objects() #infer_objects soft converts object columns to their correct types

# tempX = X.iloc[:15,:]
# tempX = convert_to_data(tempX,datadf,datacols)
# print(tempX.dtypes)

X = convert_to_data(X,datadf,datacols)
print(X.dtypes)

C:\Users\Hugh\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Hugh\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Player1                                      object
Player2                                      object
Rank_diff                                   float64
ytd_Aces%_Stddev_diff                       float64
ytd_ServesWon%_Stddev_diff                  float64
careeravg_Sets_Won_diff                     float64
careeravg_Games_Won_diff                    float64
careeravg_Aces_diff                         float64
careeravg_DoubleFaults_diff                 float64
careeravg_FirstServes_Won_diff              float64
careeravg_FirstServes_In_diff               float64
careeravg_SecondServes_Won_diff             float64
careeravg_SecondServes_In_diff              float64
careeravg_BreakPoints_Won_diff              float64
careeravg_BreakPoints_diff                  float64
careeravg_ReturnPoints_Won_diff             float64
careeravg_ReturnPoints_Faced_diff           float64
careeravg_TotalPoints_Won_diff              float64
careeravg_won_game?_diff                    float64
careeravg_Fi

In [4]:
#training and then testing a basic model
from sklearn.model_selection import GridSearchCV
fullX = X.copy()
# X.drop(['Player1','Player2','datetime'],axis=1,inplace=True)
Xtrain,Xtest,ytrain,ytest = train_test_split(X,Y,test_size=.3)
params ={'max_depth':[4,5,6],
         'n_estimators':[100,200,300],
         'learning_rate':[.01,.03,.1]
        }
estimator = xgb.XGBClassifier()
tuneparam = GridSearchCV(estimator,params,n_jobs=-1)
print("tuning model")
tuneparam.fit(Xtrain,ytrain)

model = tuneparam.best_estimator_
best_params = tuneparam.best_params_
def get_scores(predy,ytest):
    import sklearn.metrics as metrics
    acc = metrics.accuracy_score(ytest,predy)
    prec = metrics.precision_score(ytest,predy)
    recall = metrics.recall_score(ytest,predy)
    auc = metrics.roc_auc_score(ytest,predy)
    return {'accuracy':acc,'precision':prec,'recall':recall,'auc':auc}
score = model.score(Xtest,ytest)
predy = model.predict(Xtest)

scoresdict = get_scores(predy,ytest)
print("="*200)
print("Score: {}".format(score))
for key,value in scoresdict.items():
    print("{}: {}".format(key,value))

print("="*200)
for idx, col in enumerate(Xtrain.columns.tolist()):
    print("{} importance: {}".format(col,model.feature_importances_[idx]))


tuning model


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\Hugh\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\Hugh\Anaconda3\lib\runpy.py i

What about a model just trained using Ranked Diff? (curious to see how much predictive power rank has)

In [ ]:
rankXtrain = Xtrain.loc[:,'Rank_diff'].values.reshape(Xtrain.shape[0],1)
rankXtest = Xtest.loc[:,'Rank_diff'].values.reshape(Xtest.shape[0],1)

# print(rankXtrain.shape)
params ={'max_depth':[4,5,6],
         'n_estimators':[100,200,300],
         'learning_rate':[.01,.03,.1]
        }
estimator = xgb.XGBClassifier()
tuneparam = GridSearchCV(estimator,params,n_jobs=-1)
print("tuning model")
tuneparam.fit(rankXtrain,ytrain)

model2 = tuneparam.best_estimator_
best_params2 = tuneparam.best_params_

score2 = model2.score(rankXtest,ytest)
predy2 = model2.predict(rankXtest)
scoresdict2 = get_scores(predy2,ytest)

print("="*200)
print("Score: {}".format(score2))
for key,value in scoresdict2.items():
    print("{}: {}".format(key,value))

print("="*200)
print("{} importance: {}".format(col,model2.feature_importances_))


In [ ]:
print(best_params)
print(best_params2)